# Initial definitions

In [ ]:
%env HADOOP_VERSION     2.9.2
%env HADOOP_HOME hadoop-2.9.2

# Preparing the environment

## Downloading Hadoop

In [ ]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/hadoop-${HADOOP_VERSION}/hadoop-${HADOOP_VERSION}.tar.gz -q --show-progress

## Extracting compressed files and removing .tar

In [ ]:
!tar -xvf hadoop-${HADOOP_VERSION}.tar.gz >/dev/null 
!rm       hadoop-${HADOOP_VERSION}.tar.gz

## Discovering the Java path

In [ ]:
!dirname $(dirname $(readlink -f $(which javac)))

## Setting the Java path envvar

We also added it to user's .bashrc so it will be loaded as the nodes perform ssh connections.

In [ ]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

In [ ]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/.bashrc
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64" >> ~/.profile
!echo "export HADOOP_HOME=$(pwd)/$HADOOP_HOME"             >> ~/.bashrc
!echo "export HADOOP_HOME=$(pwd)/$HADOOP_HOME"             >> ~/.profile

!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " >> ${HADOOP_HOME}/etc/hadoop/hadoop-env.sh


# Hadoop in Pseudo-Distributed Mode

## Preparing the environment

### Starting sshd server

Check `/binder/postBuild` and `/resources/configs/ssh/sshd_config` files for more details

In [ ]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding names to know hosts 

Commands below stablish ssh connections to used host names/ips. This step avoids yes/no host confirmation.

In [ ]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Adding ssh options to Hadoop via envvar

* connecting in a diferent port (`-p 8822`)
* avoiding host key checking (`-o StrictHostKeyChecking=no`)

In [ ]:
%env HADOOP_SSH_OPTS= -o StrictHostKeyChecking=no -p 8822

In [ ]:
%env PDSH_RCMD_TYPE ssh

### Copying configurations files to Hadoop folder

Check the configuration files accordingly to the Hadoop version. 
Refer to the `/resources/configs/hadoop/<version>`.

In [ ]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/core-site.xml   ${HADOOP_HOME}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/hdfs-site.xml   ${HADOOP_HOME}/etc/hadoop/

## Formatting the filesystem

In [ ]:
!${HADOOP_HOME}/bin/hdfs namenode -format -force -nonInteractive

## Starting DFS (NameNode, SecondaryNameNode, and DataNode daemons)

In [ ]:
!${HADOOP_HOME}/sbin/start-dfs.sh
!jps

### Creating folders in the distributed file system

In [ ]:
!${HADOOP_HOME}/bin/hdfs dfs -mkdir /user/

### Listing files in a folder of the distributed file system

In [ ]:
!${HADOOP_HOME}/bin/hdfs dfs -ls /

# Starting YARN in Pseudo-Distributed Mode

## Preparing the environment

### Copying configurations files to Hadoop folder

In [ ]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/mapred-site.xml ${HADOOP_HOME}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/yarn-site.xml   ${HADOOP_HOME}/etc/hadoop/

## Starting YARN

In [ ]:
!${HADOOP_HOME}/sbin/start-yarn.sh
!jps

# Creating Hive HDFS folders

In [ ]:
!./$HADOOP_HOME/bin/hadoop fs -mkdir       /tmp
!./$HADOOP_HOME/bin/hadoop fs -mkdir       /user/hive/
!./$HADOOP_HOME/bin/hadoop fs -mkdir       /user/hive/warehouse
!./$HADOOP_HOME/bin/hadoop fs -chmod g+w   /tmp
!./$HADOOP_HOME/bin/hadoop fs -chmod g+w   /user/hive/warehouse